In [1]:
from argparse import Namespace

import torch
import numpy as np
import sys
import os
import dlib
from pathlib import Path

from PIL import Image


from models.Embedding import Embedding
from models.Alignment import Alignment
from models.Blending import Blending

from utils.drive import open_url
from utils.shape_predictor import align_face

import torchvision

In [2]:
align_args = Namespace(unprocessed_dir = 'unprocessed', output_dir = 'input/face', output_size = 1024,
 cache_dir = 'cache', inter_method = 'bicubic')

In [3]:
#Loading Shape Predictor for Alignment
cache_dir = Path(align_args.cache_dir)
cache_dir.mkdir(parents=True, exist_ok=True)

output_dir = Path(align_args.output_dir)
output_dir.mkdir(parents=True,exist_ok=True)

print("Downloading Shape Predictor")
f=open_url("https://drive.google.com/uc?id=1huhv8PYpNNKbGCLOaYUjOgR1pY5pmbJx", cache_dir=cache_dir, return_path=True)
predictor = dlib.shape_predictor(f)

In [4]:
#Automatic Image Cropper

name: str = 'test'

img = Image.open(f'unprocessed/{name}.jpg')

img = img.crop((448, 28, 1472, 1052))
img.save(f'unprocessed/{name}.jpg')

In [8]:
def align_face(align_args):    
    for im in Path(align_args.unprocessed_dir).glob("*.*"):
        faces = align_face(str(im),predictor)

        for i,face in enumerate(faces):
            if(align_args.output_size):
                factor = 1024//align_args.output_size
                assert align_args.output_size*factor == 1024
                face_tensor = torchvision.transforms.ToTensor()(face).unsqueeze(0).cuda()
                face_tensor_lr = face_tensor[0].cpu().detach().clamp(0, 1)
                face = torchvision.transforms.ToPILImage()(face_tensor_lr)
                if factor != 1:
                    face = face.resize((align_args.output_size, align_args.output_size), Image.LANCZOS)
            if len(faces) > 1:
                face.save(Path(align_args.output_dir) / (im.stem+f"_{i}.png"))
            else:
                face.save(Path(align_args.output_dir) / (im.stem + f".png"))

Maya.jpg: Number of faces detected: 1


In [7]:
# I/O arguments
ref: str = 'image.png'
tgt: str = f'{id}.png'

args = Namespace(input_dir='input/face', output_dir='output', im_path1 = ref, im_path2=tgt, im_path3= tgt,
 sign='realistic', smooth=5, size=1024, ckpt='pretrained_models/ffhq.pt', channel_multiplier=2,
  latent=512, n_mlp=8, device='cuda', seed=None, tile_latent=False, opt_name='adam',
   learning_rate=0.01, lr_schedule='fixed', save_intermediate=False, save_interval=300,
    verbose=False, seg_ckpt='pretrained_models/seg.pth', percept_lambda=1.0, l2_lambda=1.0,
     p_norm_lambda=0.001, l_F_lambda=0.1, W_steps=250, FS_steps=250, ce_lambda=1.0, style_lambda=40000.0,
      align_steps1=100, align_steps2=100, face_lambda=1.0, hair_lambda=1.0, blend_steps=200)

In [8]:
im_path1 = os.path.join(args.input_dir, args.im_path1)
im_path2 = os.path.join(args.input_dir, args.im_path2)

In [9]:
#Embed input image
ii2s = Embedding(args)

im_set = {im_path1}
ii2s.invert_images_in_W([*im_set])
ii2s.invert_images_in_FS([*im_set])



Loading StyleGAN2 from checkpoint: pretrained_models/ffhq.pt
Setting up Perceptual loss...
Loading model from: /mnt/d/Programming/Spectrum/Barbershop/Barbershop/losses/lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
Number of images: 1


Images: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:57<00:00, 57.61s/it]


Number of images: 1


Images: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:48<00:00, 48.87s/it]


: 

In [43]:
#Alignment Step

align = Alignment(args)
align.align_images(im_path1, im_path2, sign=args.sign, align_more_region=False, smooth=args.smooth)

#originalimage_chosenimage.png

Loading StyleGAN2 from checkpoint: pretrained_models/ffhq.pt


Align Step 2:   0%|                                                                             | 0/100 [00:00<?, ?it/s]/mnt/d/Programming/miniconda3/envs/Barbershop/lib/python3.7/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/mnt/d/Programming/miniconda3/envs/Barbershop/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for detail

In [13]:
blend = Blending(args)
blend.blend_images(im_path1, im_path2, im_path2, sign=args.sign)
#originalimage_chosenimage_chosenimage.png

Loading StyleGAN2 from checkpoint: pretrained_models/ffhq.pt
Setting up Perceptual loss...
Loading model from: /mnt/d/Programming/Spectrum/Barbershop/Barbershop/losses/masked_lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
Setting up Perceptual loss...
Loading model from: /mnt/d/Programming/Spectrum/Barbershop/Barbershop/losses/masked_lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
